# Model Training Notebook

In [1]:
%matplotlib inline
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from datetime import datetime
from IPython.core.display import display, HTML
import cv2
from PIL import Image
from pathlib import Path
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import ast

from sklearn.model_selection import train_test_split

# repeatable functions housed in the utils file and imported here
from utils import *
from model_training_utils import train as training_loop
from model_training_utils import validate as validation_loop
from model_training_utils import hp_grid_search
from models import CNN, YOLO, VGG, PreTrainedVGG, SimpleVGG, SingleLinear

## Create Training and Validation Datasets

In [2]:
df= pd.read_csv('./datasets/annotations_map.csv', converters={'new_bb': from_np_array})

In [3]:
df_train = df.reset_index()
X = df_train[['new_path','new_bb']]
Y = df_train['class']
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.25, random_state=42)
print('Number of training observations: ', X_train.shape[0])
print('Number of validation observations: ', X_val.shape[0])

Number of training observations:  32
Number of validation observations:  11


## Build Simple CNN

In [4]:
simple_cnn = CNN()

## Build VGG
Implementation of VGG-16 architecture based on https://neurohive.io/en/popular-networks/vgg16/. VGG-16 is known for its high accuracy and speen on object detection tasks, largely attributed to its 3x3 kernel size.

In [5]:
vgg = VGG()

## Pre-trained VGG
A pytorch VGG-16 model that was pre-trained on object identification. Utilized for benchmarking our own implementations against.

In [6]:
preTrainedVGG = PreTrainedVGG()

## Build Simple VGG
Our simplified VGG based model that attempts to achieve better performance by using less layers. The idea behind this was that the many layers and transformations in VGG can cause it to struggle with smaller objects.

In [7]:
simple_vgg = SimpleVGG()

## Build YOLO

#### The Design of the YOLO NN was taken from the following paper:

https://arxiv.org/pdf/1506.02640.pdf - "You Only Look Once: Unified, Real-Time Object Detection" by Redmon, Divvala, Girshick, and Farhadi

The following article is YOLO V2:
https://arxiv.org/pdf/1612.08242v1.pdf - "YOLO 9000: Better, Faster, Stronger" by Redmon, and Farhadi

In [8]:
yolo = YOLO()

# Reference Model

In [4]:
from torchvision import transforms

In [5]:
inception_v3 = nn.Sequential(torchvision.models.inception_v3(pretrained = True, aux_logits=False), SingleLinear())

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /Users/andrew.lofgreen/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

# Train the Model


- Structured similarly to main.py file from pytorch part of A2

In [6]:
## Define Hyperparameters -- Currently setting values that we can modify
loss_type = "l1"
learning_rate = 0.001
momentum = 0.5
reg = 1e-2

training_batch_size = 40
validation_batch_size = 40

model_type = "Inception"

In [7]:
if loss_type == "l1":
    criterion = nn.L1Loss()
    
if loss_type == "l2":
    criterion = nn.MSELoss()
    
if model_type == "SimpleCNN":
    model = simple_cnn
elif model_type == "YOLO":
    model = yolo
elif model_type == "VGG":
    model = vgg
elif model_type == "PreTrainedVGG":
    model = preTrainedVGG
elif model_type == "SimpleVGG":
    model = simple_vgg
elif model_type == "Inception":
    model = inception_v3
    
train_ds = WaldoDataset(X_train['new_path'],X_train['new_bb'] ,y_train)
valid_ds = WaldoDataset(X_val['new_path'],X_val['new_bb'],y_val)

train_dl = DataLoader(train_ds, batch_size=training_batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=validation_batch_size)

optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                            momentum=momentum,
                            weight_decay=reg)

In [ ]:
training_loop(model_type=model_type, model= model, optimizer = optimizer, train_dl= train_dl, valid_dl=valid_dl, epochs= 20, criterion= criterion, verbose= True, return_loss= False)

/Users/andrew.lofgreen/Documents/OMSCS/cs7643/dl7643/CS7643-Final-Project/model_training_utils.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_bb= torch.tensor(y_bb)


 
--------------------------------------------------------
Training Loss for Epoch 0: 6.833187103271484


/Users/andrew.lofgreen/Documents/OMSCS/cs7643/dl7643/CS7643-Final-Project/model_training_utils.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_bb= torch.tensor(y_bb)


Validation Loss for Epoch 0: 20.37358925559304
 
--------------------------------------------------------
Training Loss for Epoch 1: 6.831632614135742
Validation Loss for Epoch 1: 20.368273648348723
 
--------------------------------------------------------
Training Loss for Epoch 2: 6.831934928894043
Validation Loss for Epoch 2: 20.36471141468395
 
--------------------------------------------------------
Training Loss for Epoch 3: 6.830149173736572
Validation Loss for Epoch 3: 20.3606220592152
 
--------------------------------------------------------
Training Loss for Epoch 4: 6.829981803894043
Validation Loss for Epoch 4: 20.35437150435014
 
--------------------------------------------------------
Training Loss for Epoch 5: 6.827932357788086
Validation Loss for Epoch 5: 20.349197387695312
 
--------------------------------------------------------
Training Loss for Epoch 6: 6.827206611633301
Validation Loss for Epoch 6: 20.344254927201703
 
-------------------------------------------

In [ ]:
loss_type= ["l1"]
learning_rate= [0.001,0.0001]
momentum = [0.9]
reg = [0.01]
batch_size= [10]

all_training_loss, all_validation_loss= hp_grid_search(model_type= "SimpleCNN", 
               lr_list=learning_rate, 
               momentum_list=momentum, 
               reg_list=reg, 
               batch_size_list=batch_size,
               train_ds= train_ds,
               valid_ds= valid_ds,
               optimizer= optimizer, 
               loss_type_list=loss_type,
               epochs= 10,
               save_all_plots="Yes", 
               save_final_plot="Yes",
               final_plot_prefix="Test", 
               return_all_loss= True)

## Generate GradCAM
Generates GradCAM images for Simple CNN and saves them to images/SimpleCNNResizedGradCam

In [ ]:
grad_cam(model_type=model_type, model=model, train_dl=train_dl)